<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0925RNN_3twilight_poetries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
filename: 2022_0925RNN_3twilight_poetries.ipynb

---

# 三夕の歌 (寂蓮，西行，定家) を学習するリカレントニューラルネットワーク

## 目次

1. 語彙辞書の作成
2. 埋草処理
3. ワンホット符号化
4. 言語モデルの定義
5. 言語モデルの訓練の実施
6. 学習結果の評価


In [ ]:
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install ipynbname --upgrade
    !git clone https://github.com/ShinAsakawa/bit.git
    import bit  
isColab = bit.isColab
HOME = bit.HOME

# 1 語彙辞書の作成

In [2]:
import torch
from torch import nn
import numpy as np

twilight_poetries = ['さびしさは 其の色としも なかりけり まき立つ山の 秋の夕暮',  #寂蓮
                     '心なき 身にもあはれは しられけり 鴫立つ沢の 秋の夕暮',     #西行
                     'み渡せば 花ももみぢも なかりけり 浦の苫屋の 秋の夕暮'      #定家
                    ]

# twilight_poetries では変数名として長いから，タイプミスしやすい。そこで名前を変更
data = twilight_poetries

# 3 つ短歌を結合し，結合した文章からユニークな文字を抽出
tokens = sorted(set(''.join(data)))

# トークン ID 番号から文字を返す辞書
idx2tkn = dict(enumerate(tokens))

# 文字からトークン ID を返す辞書
tkn2idx = {ch:idx for idx, ch in idx2tkn.items()}

In [ ]:
print(idx2tkn)
print(tkn2idx)

次に，すべての文がサンプルの長さになるように，入力文のパディングを行います。
RNN は通常，様々なサイズの入力を取り込むことができますが，通常は，学習処理を高速化するために，学習データを一括して送り込みたいと考えるでしょう。
バッチを使用してデータを学習するためには，入力データ内の各系列が同じサイズ (同一系列長) であることを確認する必要があります。
<!-- Next, we'll be padding our input sentences to ensure that all the sentences are of the sample length. 
While RNNs are typically able to take in variably sized inputs, we will usually want to feed training data in batches to speed up the training process. -->

そのため，多くの場合，短すぎる配列は 埋め草 ID 例えば  **0** で埋め，長すぎる配列は切り捨てることで，パディングを行うことができます。
今回の場合は，最も長い配列の長さを求め，その長さに合わせて残りの文章を空白でパディングすることにします。
<!-- In order to used batches to train on our data, we'll need to ensure that each sequence within the input data are of equal size. -->

したがって，ほとんどの場合，パディングは短すぎる配列を **0** の値で埋め，長すぎる配列を切り詰めることで行うことができます。
今回は，最も長い配列の長さを求め，その長さに合わせて残りの文章を空白でパディングすることにします。
<!-- Therefore, in most cases, padding can be done by filling up sequences that are too short with **0** values and trimming sequences that are too long. 
In our case, we'll be finding the length of the longest sequence and padding the rest of the sentences with blank spaces to match that length. -->

In [ ]:
maxlen = len(max(data, key=len))
print(f"最長文字列: {maxlen} 文字")

# 2 埋め草処理

In [ ]:
# パディング
# 文のリストを繰り返して，文長さが最長文長に一致するまで空白文字 ‘ ' を追加する単純な繰り返し
for i in range(len(data)):
    while len(data[i]) < maxlen:
        data[i] += ' '

for datum in data:
    print(f'-{datum}-')

各時刻で次字を予測するため，各歌を以下ように分解します

- 入力データ: 最後の入力文字はモデルに入れる必要がないため除外する
- 標的/正解ラベル: 入力データより 1 時刻前の時間。これが入力データに対応する各時刻でのモデルの「正解」となる

In [ ]:
# 入力配列とターゲット配列を格納するリストの作成
inputs_seq = []
target_seq = []

for i in range(len(data)):
    # 入力系列の最後の文字を削除
    inputs_seq.append(data[i][:-1])
    
    # 標的配列の最初の文字を削除
    target_seq.append(data[i][1:])
    print(f"入力系列: {inputs_seq[i]}")
    print(f"目標系列: {target_seq[i]}")

# 3 ワンホット符号化

ここで，入力配列と標的配列を，上で作成した辞書を使って写像することで，文字ではなく，整数の配列に変換することができます。
これによって，入力系列をワンホットベクトルへ符号化できるようになります。


In [ ]:
inputs_ids, target_ids = [], []  # トークン ID を入れておくリストを用意
for i in range(len(data)):
    inputs_ids.append([tkn2idx[ch] for ch in inputs_seq[i]])
    target_ids.append([tkn2idx[ch] for ch in target_seq[i]])
    
print(inputs_ids)
print(target_ids)

入力配列をワンホットベクトルに符号化する前に，3 つの重要な変数を定義しておきます。

- **dict_size**: テキストに含まれるユニークな文字の数。
各文字がそのベクトル内の割り当てられたインデックスを持つように，ワンホットベクトルのサイズを決定します。
- **seq_len**: モデルに入力する配列の長さ。
全ての文章の長さを最長の文章と同じになるように標準化したので，この値は最後の文字の入力を削除したため，最大の長さ - 1 となる
- **batch_size**: バッチとして定義され，モデルに投入される文の数


In [8]:
dic_size = len(tkn2idx)
seq_len = maxlen - 1
batch_size = len(data)

def one_hot_encode(seq:list,
                   dic_size:int, 
                   seq_len:int, 
                   batch_size:int):
    # すべての要素が 0 である，3 元テンソルを定義
    ret = np.zeros((batch_size, seq_len, dic_size), dtype=np.float32)
    
    # 上で定義した 3 元テンソルに対して該当するトークン ID の要素を 1 にする
    for batch in range(batch_size):
        for t in range(seq_len):
            ret[batch, t, seq[batch][t]] = 1
    return ret

In [9]:
inputs = one_hot_encode(seq=inputs_ids, 
                        dic_size=dic_size, 
                        seq_len=seq_len, 
                        batch_size=batch_size)
print(f"入力データ形状: {inputs.shape} --> (バッチサイズ, 系列長, ワンホット埋め込みベクトル次元)")

入力データ形状: (3, 29, 39) --> (バッチサイズ, 系列長, ワンホット埋め込みベクトル次元)


データの前処理はすべて終わったので，次はデータを numpy の配列から PyTorch 独自のデータ構造である **Torch Tensors** に変換します。

In [10]:
inputs_seq = torch.from_numpy(inputs)
target_seq = torch.Tensor(target_ids)

# 4 言語モデルの定義
Torch ライブラリを使ってモデルを定義していきます。
ここで，完全連結層，畳み込み層，バニラ RNN 層，LSTM 層，その他いろいろな層を追加したり削除したりすることができます! 

モデルの構築を始める前に，PyTorch のビルドイン機能を使って，実行しているデバイス (CP Uか GPU か) を確認してみましょう。
この実装では，学習が本当に簡潔なので，GPU は必要ありません。
しかし，大規模なデータセットや数百万の学習可能なパラメータを持つモデルに進んでいくと，GPU を使うことは学習を高速で進めることができるようになります。

In [ ]:
# もし GPU が利用可能であれば，デバイスを GPU に設定します。
# このデバイス変数は後ほどコード内で使用します。
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'device:{device}')

独自のニューラルネットワークモデルの構築を開始するには、すべてのニューラルネットワークモジュールのための PyTorch の基本クラス (`nn.module`) を継承するクラスを定義することができます。
その後，コンストラクタで変数とモデルの層を定義します。
このモデルでは，RNN の 1 層と全連結層を使用します。
全連結層は，RNN の出力を希望する出力形状に変換する役割を担います。

また，クラスメソッドとして `forward()` の下にフォワードパス関数を定義する必要があります。
`forward()`  関数は順番に実行されるので，入力とゼロ初期化された隠れ状態をまず RNN 層に渡し，その後に RNN の出力を全連結層に渡すことになります。
コンストラクタで定義した層を使っていることに注意してください。


最後に定義するのは，先ほど隠れ状態を初期化するために呼び出したメソッド，`init_hidden()` です。
これは基本的に，隠れ層の形をしたゼロのテンソルを作成します。

In [12]:
import torch.nn as nn

class RNN_Model(nn.Module):
    """このメソッドは，PyTorch 公式サイトにあるサンプルコードをわかりやすく書き換えたもの。
    メソッドを実体化 (インスタンス化) する際には，4 つの整数引数と 1 つの文字列引数を指定します。
    整数とは次の 4 つ: input_size, output_size, hidden_size, num_layers, 
    文字列引数とは `rnn_type`  であり，`rnn_type` には `LSTM`, `GRU`, `RNN_TANH`, `RNN_RELU` の
    いずれかが指定できる。
    
    このメソッドを呼び出す際には，入力テンソル (`torch.tensor`) を与える。
    このとき，入力テンソルの形状 (サイズ) は [`バッチ`, `系列`, `データ`] である必要がある。
    PyTorch の実装では，`batch_first` オプションにより，入力テンソルタの第 1 次元が，系列かバッチかを選択可能である。
    駄菓子菓子，ここでは `batch_first` を決め打ちしている
    """
    def __init__(self, 
                 input_size:int, 
                 output_size:int, 
                 hidden_size:int, 
                 num_layers:int=1,
                 rnn_type:str='RNN_TANH',
                 dropout:float=0.,
                 device="cuda" if torch.cuda.is_available() else "cpu"
                ):
        
        super().__init__()

        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(torch.nn, rnn_type)(
                input_size=input_size, 
                hidden_size=hidden_size, 
                num_layers=num_layers, 
                batch_first=True,
                dropout=dropout).to(device)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 
                                'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """rnn_type で指定可能なモデルは ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU'] です""")
            self.rnn = torch.nn.RNN(
                input_size=input_size, 
                hidden_size=hidden_size, 
                num_layers=num_layers, 
                nonlinearity=nonlinearity, 
                batch_first=True,
                dropout=dropout).to(device)

        self.rnn_type = rnn_type
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # 全結合層
        self.fc = nn.Linear(
            in_features=hidden_size, 
            out_features=output_size)
    
    def forward(self, 
                x:torch.Tensor=None):
        
        # batch_first を仮定しているので 0 次元目がミニバッチ長を表す
        batch_size = x.size(0)  

        #以下で定義するメソッドを使用して、最初の入力に対して隠れ層状態を初期化
        hidden = self.init_hidden(batch_size)

        # 入力と隠れ層状態をモデルに渡して出力を得る
        out, hidden = self.rnn(x, hidden)
        
        # 全結合層に収まるように出力を整形
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, 
                    batch_size:int=1):
        """順向パスで使用する時刻 0 の隠れ層を生成
        上で指定したデバイスに隠れ層の状態を保持したテンソルを返す"""
        
        if self.rnn_type == 'LSTM':
            cell = torch.zeros(self.num_layers, 
                               batch_size, 
                               self.hidden_size).to(device)
            hidden = torch.zeros(self.num_layers, 
                                 batch_size, 
                                 self.hidden_size).to(device)
            return cell, hidden

        else:
            hidden = torch.zeros(self.num_layers, 
                                 batch_size, 
                                 self.hidden_size).to(device)
            return hidden
    
# test_model = RNN_Model(input_size=3, output_size=2, hidden_size=5)

上記のモデルを定義した後，関連するパラメータでモデルを実体化し，同様にハイパーパラメータを定義する必要があります。
ハイパーパラメータは以下のように定義します。
<!-- After defining the model above, we'll have to instantiate the model with the relevant parameters and define our hyperparamters as well. 
The hyperparameters we're defining below are:-->

- **`n_epochs`**:  エポック数。モデルが訓練データセット全体を通過する回数
- **`lr`**: 学習率。誤差逆伝播による学習が行われるたびに，モデルが結合係数を更新する率。
    - 学習率が小さいと，モデルはより小さな大きさで重みの値を変更することを意味する
    - 学習率が大きいと，各時刻で重みがより大きく更新されることを意味する。

<!-- - *n_epochs*: Number of Epochs -- This refers to the number of times our model will go through the entire training dataset
- *lr*: Learning Rate -- This affects the rate at which our model updates the weights in the cells each time backpropogation is done
    - A smaller learning rate means that the model changes the values of the weight with a smaller magnitude
    - A larger learning rate means that the weights are updated to a larger extent for each time step -->
    
他のニューラルネットワークと同様，オプティマイザと損失関数を定義する必要があります。
最終的な出力は基本的に分類課題なので，`CrossEntropyLoss` を使用することにします。
<!-- Similar to other neural networks, we have to define the optimizer and loss function as well. We’ll be using CrossEntropyLoss as the final output is basically a classification task. -->

In [ ]:
# 言語モデルを実体化
model = RNN_Model(
    rnn_type = 'LSTM',  # or choose among `RNN_TANH`, `RNN_RELU`, `GRU`, and `LSTM`
    input_size=dic_size, 
    output_size=dic_size, 
    hidden_size=8,     # 任意の整数に変更可能なハイパーパラメータ
    num_layers=1)

# モデルを予め定めておいたデバイスに設定 (`cpu` または `cuda`)
model = model.to(device)

# ハイパーパラメータの定義
n_epochs=500  # エポック数
lr=0.01       # 学習率

# 損失関数と最適化手法の定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.eval()

# 5 訓練 (学習) の実施

In [ ]:
model.train()
for epoch in range(1, n_epochs + 1):
    
    optimizer.zero_grad()  # 前エポックから残存する勾配を 0 で初期化
    
    inputs_seq = inputs_seq.to(device)
    output, hidden = model(inputs_seq)
    output = output.to(device)
    
    target_seq = target_seq.to(device)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward()  # 誤差逆伝播を行って勾配を計算
    optimizer.step() # 勾配の計算に従って結合係数を更新
    
    if epoch % (n_epochs >> 2) == 0:
        print(f'エポック: {epoch:5d}/{n_epochs:5d} :',
              f'損失値: {loss.item():.3f}')

# 6 結果の評価
モデルを検査して，どのような出力が得られるか見てみましょう。
そのために，モデルの出力 ID を文字に戻すためのヘルパー関数を定義します。


In [16]:
def predict(model=nn.Module, 
            inputs:str=""):
    
    # 入力とモデルが合致するようにワンホット符号化
    _inputs_ids = np.array([[tkn2idx[x] for x in inputs]])
    inputs_ids = one_hot_encode(_inputs_ids, dic_size, _inputs_ids.shape[1], 1)
    
    # ワンホットベクトルを torch.Tensor に変換
    inputs_ids = torch.from_numpy(inputs_ids)   
    
    # デバイスに転送
    inputs_ids = inputs_ids.to(device)         
    
    # モデルの実行
    out, hidden = model(inputs_ids)
    
    # ソフトマックス関数に通して確率に変換
    probs = nn.functional.softmax(out[-1], dim=0).data

    # 出力から最も高い確率の得点を持つクラスを取り出す
    output_ids = torch.max(probs, dim=0)[1].item()

    return idx2tkn[output_ids], hidden

In [17]:
def sample(model:nn.Module, 
           out_len:int=0, 
           start:str=''):
    model.eval()
    
    # 最初に開始時刻 (t=0) の文字を実行するために設定
    chars = [ch for ch in start]
    size = out_len - len(chars)
    
    # 現時刻までに出力された文字列を渡して次の文字を取得し，付け加える
    for _ in range(size):
        out, h = predict(model=model, inputs=chars)
        chars.append(out)
        #print("".join(chars))

    return ''.join(chars)

In [ ]:
print(sample(model, 30, 'み'))
print(sample(model, 30, '心'))
print(sample(model, 30, 'さ'))

## 演習

1. 言語モデルを定義する際に, `rnn_type=` の引数で，以下の 4 種類を試せ。
`['RNN_TANH', 'RNN_RELU', 'GRU', 'LSTM']` 結果に違いが生じるか?
2. 言語モデルの中間層のニューロン数 `hidden_size=` を変化させて結果を観察せよ
3. 言語モデルの総数 `num_layers=` を変化させて結果を観察せよ
4. 学習率を変化させて見よ。結果を観察してみよ



